# 1. 문서의 내용을 읽는다
# 2. 문서를 쪼갠다
#     - 토큰수 초과로 답변을 생성못할 수 있음
#     - 문서가 길면 답변 생성이 오래걸림
# 3. 임베딩 -> 벡터 데이터베이스에 저장
# 4. 질문이 있을때, 벡터 디비에 유사도 검사
# 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달



In [1]:
%pip install -qU python-dotenv langchain-upstage langchain-community langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤때 하나의 하나의 청크가 가질수 있는 토큰수
    chunk_overlap=200, # 문서를 가져올때 1부터 10라인까지 가져오면 그후 2부터 11 라인을 가져오는 식으로 중복되는 라인수
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

185

In [27]:
document_list

KeyboardInterrupt: 

In [4]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()
# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 `vector`로 변환
embedding = UpstageEmbeddings(model='solar-embedding-1-large')


In [5]:
%pip install --upgrade --quiet \
    langchain-pinecone \
    langchain-openai \
    langchain 

Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-upstage-table-index'
pinecone_api_key = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')


In [ ]:
%pip install -qU langchain langchainhub

In [29]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/Users/r00360/.pyenv/versions/3.10.12/envs/llm-langchain/lib/python3.10/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [30]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [31]:
## QA체인 만들기

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [32]:
query = '연동 5천만원인 직장인의 소득세는 얼마인가요?'
ai_message = qa_chain({"query": query})

In [33]:
ai_message

{'query': '연동 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연소득 5천만원인 직장인의 소득세는 5천만원 이하의 과세표준에 해당하는 세율을 적용하여 계산합니다. 세율구간에 따르면, 84만원 + (1,400만원을 초과하는 금액의 15%)입니다. 따라서, 소득세는 84만원 + ((5,000만원 - 1,400만원) × 0.15) = 84만원 + 540만원 = 624만원이 됩니다.'}